In [16]:
class Links:

    #Class variables
    #links = []
    #indices = []
    #unordered_indices = []

    n_ts = {"tdc" : 47, "high" : 8, "medium" : 10, "low" : 12}    # Creating a dictionary with sl and the corresponding number of time-slots in the channel
    channel = {"tdc" : 0, "high" : 1, "medium" : 2, "low" : 3}    # A dictionary for channel and sl 
    channel_ts = {1 : 8, 2 : 10, 3 : 12}
    #channel_to_wavelength = {0 : lambda_tdc, 1 : lambda_q1, 2 : lambda_q2, 3 : lambda_q3} #won't work
    
    def __init__(self, nodes, w):    # nodes = (s, d); is a tuple so as to not confuse s and d individually
        self.nodes = nodes
        self.lambda_tdc = np.ones(47, dtype = bool)

        time_slots1 = 8
        time_slots2 = 10 
        time_slots3 = 12
        self.lambda_q1 = np.ones(time_slots1, dtype = bool)    # for high sl
        self.lambda_q2 = np.ones(time_slots2, dtype = bool)    # for medium sl    
        self.lambda_q3 = np.ones(time_slots3, dtype = bool)    # for low sl

    def update_link(self, channel, slot):
        # call : links[index].update_link(channel, ts); where ts is timeslot
        #QSC = ["lambda_q1", "lambda_q2", "lambda_q3"]
        QSC = [1, 2, 3]
        TDC = [0]
        AC = QSC + TDC
        #if channel not in AC or ts < 0:
            #print("Invalid Update request!")

        if channel == 0:
            self.lambda_tdc[slot] = False
            #print(f"The updated tdc resources for {self.nodes} are : {self.lambda_tdc}")
        elif channel == 1 and slot < len(self.lambda_q1):
            self.lambda_q1[slot] = False
            print(f"The updated q1 resources for {self.nodes} are : {self.lambda_q1}")
        elif channel == 2 and slot < len(self.lambda_q2):
            self.lambda_q2[slot] = False
            print(f"The updated q2 resources for {self.nodes} are : {self.lambda_q2}")
        elif channel == 3 and slot < len(self.lambda_q3):
            self.lambda_q3[slot] = False
            print(f"The updated q3 resources for {self.nodes} are : {self.lambda_q3}")

        else:
            print("Invalid time slot value")

    def resources_available(self, channel, ts):
        if channel not in [1, 2, 3] or ts < 0:
            print("Invalid Update request!")

        elif channel == 1 and ts < len(self.lambda_q1):
            return self.lambda_q1[ts]
            
        elif channel == 2 and ts < len(self.lambda_q2):
            return self.lambda_q2[ts]
            
        elif channel == 3 and ts < len(self.lambda_q3):
            return self.lambda_q3[ts]

        else:
            print("Invalid time slot value")

    #def allocate_resources(self)

    def ts_count(self, channel):
        
        if channel == 1:
            count = np.count_nonzero(self.lambda_q1)    # zero => utilized slots
        if channel == 2:
            count = np.count_nonzero(self.lambda_q2) 
        if channel == 3:
            count = np.count_nonzero(self.lambda_q3) 

        utilized_count = Links.channel_ts[channel] - count
        return utilized_count
        
    def display_info(self, wl_info = False):
        q1_count = np.count_nonzero(self.lambda_q1) # nonzero => Available slots
        q2_count = np.count_nonzero(self.lambda_q2) #== True
        q3_count = np.count_nonzero(self.lambda_q3) #== True
        tdc_count = np.count_nonzero(self.lambda_tdc) #== True
        print(f"Link: Nodes ={self.nodes}, lambda_tdc_count={tdc_count}, lambda_q1_count={q1_count}, lambda_q2_count={q2_count}, lambda_q3_count={q3_count}")

        if wl_info:    # To show the wavelength occupancy
            print(f" q1 : {self.lambda_q1}, q2 : {self.lambda_q2}, q3 : {self.lambda_q3}, tdc : {self.lambda_tdc}")
        
        
    @classmethod
    def initialize_links(cls, edges):

        all_values = [nodes for row in edges for nodes in row[:2]]    # A list of all the values/nodes in first 2 columns : s & d
        m = min(all_values)
        M = max(all_values)
        numNodes = max(all_values) - min(all_values) + 1     # Correction for the convention in indexing of nodes in the data of edges
    
        links = np.zeros([M+1, M+1], dtype = object)    # Will have redundant entries. M is the highest node index
        #link_indices = np.zeros_like(links, dtype = bool)

        #links = {}
        unordered_indices = []
        for (s, d, w) in edges:

            nodes = (s, d)
            link = cls(nodes, w)
            unordered_indices.append(nodes)

            # In the above call of __init__ constructor, the wavelength resources have also been initialized to all available(True)
            
            links[nodes] = link
            links[d, s] = link
                    
        indices = links != 0    # A mask that holds the location of each link. NOTE : It stores ordered pair

        cls.links = links
        cls.indices = indices
        cls.unordered_indices = unordered_indices
        
        return (links, indices, unordered_indices)

    @classmethod
    def path_resources(cls, path, sl):
        available_tdcs = [True] * cls.n_ts["tdc"]    # For traditional data channel slots
        available_ts = [True] * cls.n_ts[sl]    # Creating a base boolean array of the size corresponding to the specific CR's sl

        #print("available_ts: ", available_ts)
        
        for s, d in zip(path, path[1:]):    # Taking consecutive pairs of nodes and selecting the particular channel
            if sl == "high":
                band = links[s, d].lambda_q1
                #print(f"link : {s, d}, lambda_q1 : {band}")
            elif sl == "medium":
                band = links[s, d].lambda_q2
                #print(f" link : {s, d}, lambda_q2 : {band}")
            else:
                band = links[s, d].lambda_q3
                #print(f"link : {s, d}, lambda_q3 : {band}")

            # Checking for continuity constraints in ts of quantum channel
            available_ts = [a and b for a, b in zip(available_ts, band)]    
            available_tdcs = [a and b for a, b in zip(available_tdcs, links[s, d].lambda_tdc)]

        return available_ts, available_tdcs

        
    @classmethod
    def FF(cls, path, cr):
        # First check for availability of resources
        # We've not considered ASLC and Resource usage extent here]

        sl = cr.sl

        available_ts, available_tdcs = cls.path_resources(path, sl)
        
        if cr.tk > len(available_ts):    # ts is 1 in our case by default. But in case we change it, this line will be useful
            raise ValueError("CR duration cannot be longer than the available time-slots in a channel")

        for i, ts in enumerate(available_ts) :
            if ts:
                for s, d in zip(path, path[1:]):    # A loop to update all the links in the path 
                    link = links[s, d]

                    # For traditional wavelength allocation
                    tdc = -1
                    for j, slot in enumerate(available_tdcs) :    # Fetching a available slot on tdc
                        if slot :
                            tdc = j    # Not allocating here because need to check for qsc as well
                            break
                    if tdc == -1:    # If no tdc slot was assigned
                        return False
                        
                    #link.lambda_tdc[tdc] = False    # Reserving the tdc slot        
                    link.update_link(cls.channel["tdc"], tdc)
                    link.update_link(cls.channel[sl], i)    # Updating the i_th ts in channel for sl to False
 
                cr.update_status("allocated")
                print(f"allocated ts {i} successfully to CR {cr.index}")
                    
                return True
                
        cr.update_status("blocked")    # If return statement of the loop wasn't executed
        return False

    @classmethod
    def TUR(cls):    # Only for quantum channel
        
        util_ts = [0, 0, 0, 0]
        tur = util_ts
        link_ts = sum(cls.n_ts[priority] for priority in ["high", "medium", "low"])    # ts in each link. In the current case, it's 8+10+12 = 30
        print("ts in 1 link : ", link_ts)
        
        print("Number of links : ", len(cls.links)) 
        
        total_ts = len(cls.links) * link_ts
        print("total_ts : ", total_ts)
        
        for nodes in cls.unordered_indices :
            link = cls.links[nodes]
            
            for i in range(3):
                util_ts[i+1] += link.ts_count(i+1)

        util_ts[0] = sum(util_ts[i+1] for i in range(3))
        print(" The number of utilized time-slots is : ", util_ts)

        tur[0] = util_ts[0]/total_ts
        for i in range(3):
            tur[i+1] = util_ts[i+1]/(len(cls.links)*cls.channel_ts[i+1])
        print("\n The time-slot utilization ratio(TUR) is : ", tur)
        
        return tur
                
    
    @classmethod
    def display_all_links(cls, wl_info = False):
        for nodes in cls.unordered_indices :
            cls.links[nodes].display_info(wl_info)


In [ ]:
#print(indices)

In [ ]:
#print(links)

In [ ]:
##for index in indices: # ERROR : Because indices isn't an array. See below cell for correction
  ##  links[index].display_info()